In [41]:
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib import style
style.use('seaborn-talk')
from matplotlib import cm
import numpy as np
import os
import scipy.io
import seaborn as sb
from scipy.fftpack import fft
import csv
import pandas as pd
from scipy.signal import butter, filtfilt
import glob
import seaborn as sns
import scikit_posthocs as sp
from scipy import signal

In [42]:
# Pathways
outpath = r'C:\Users\Daniellab\Desktop\Jorge_Analysis\Tortuosity\NewMethod'
figpath = r'C:\Users\Daniellab\Desktop\Jorge_Analysis\Tortuosity\NewMethod\Figures'
source = r'C:\Users\Daniellab\Desktop\Jorge_Analysis\Centroids_Final\Raw_NG'

In [43]:
def removeNaN(df): 
    if np.any(df.X_position.isnull()):
        a = np.asarray(np.where(df.X_position.isnull())).ravel()
        b = a[1:]
        c = a[:-1]
        d = b-c
        last_chunk = len(df.X_position) - a[-1]


        if last_chunk > 100:

            start = a[-1]+1
            end = a[-1] + last_chunk-1

        elif a[0] > 100:
            start = 0
            end = a[0]-1


        elif d.max() > 100:
            e = d.argmax()
            start = a[e]+1
            end = a[e+1]-1

        else:
            print('error')
    
        x_no_nan = df.X_position.values[start:end]
        y_no_nan = df.Y_position.values[start:end]
        return(x_no_nan, y_no_nan)
    else:
        x = df.X_position.values
        y = df.Y_position.values
        return(x,y)

In [44]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a
def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data, padlen = 2 * max(len(a), len(b)))
    return y

In [45]:
def gaussian(x,y):
    gauss = signal.gaussian(7,3)
    x_gauss = signal.convolve(x, gauss, mode='same', method = 'direct') / gauss.sum()
    y_gauss = signal.convolve(y, gauss, mode = 'same', method = 'direct') / gauss.sum()
    return(x_gauss, y_gauss)

In [46]:
def tortuosity_difference(x,y,x_smooth,y_smooth):
    x = x[3:-2]
    y = y[3:-2]
    x_smooth = x_smooth[3:-2]
    y_smooth = y_smooth[3:-2]
    i = 0
    d_list = []
    for element in x:
        dist = np.sqrt((x[i]-x_gauss[i])**2 + (y[i]-y_gauss[i])**2)
        i+=1
        d_list.append(dist)    
    mean_d = np.mean(d_list)
    std_d = np.std(d_list)
    return(mean_d, std_d)

In [47]:
def tortuosity_ratio(x,y,x_smooth, y_smooth):
    i =1
    L_list = []
    for element in x: 
        if i < len(x):
            L_ = np.sqrt((x[i]-x[i-1])**2 + (y[i]-y[i-1])**2)
            i+=1
            L_list.append(L_)
    L = np.sum(L_list)
    i = 1
    L_prime_list = []
    for element in x_smooth:
        if i < len(x_smooth):
            L_prime_ = np.sqrt((x_smooth[i]-x_smooth[i-1])**2 + (y_smooth[i]-y_smooth[i-1])**2)
            i+=1
            L_prime_list.append(L_prime_)
    L_prime = np.sum(L_prime_list)
    t_r = L/L_prime
    return(t_r)

In [48]:
csv_list = glob.glob(source + '\*Centroid*.csv')
print(len(csv_list))

54


In [49]:
t_list = []
mean_list = []
std_list = []
visit_list = []
e = 1
for element in csv_list:
    df = pd.read_csv(element)
   
    visit = df.visit_number[0]
    x, y = removeNaN(df)
    
    x_gauss, y_gauss = gaussian(x,y)
    
    
    t = df.loc[0:len(x),'FrameNumber']/100
    Time = t[len(t)-1]-t[0]
    
    
    x_filtered = butter_lowpass_filter(x, 20, len(x)/Time, order = 5)
    y_filtered = butter_lowpass_filter(y, 20, len(y)/Time, order = 5)

    if len(x) > 200:
        print(element[81:-4])
        print(len(x))
        e+=1
    mean, std = tortuosity_difference(x,y,x_filtered, y_filtered)
    
    t = tortuosity_ratio(x,y,x_filtered,y_filtered)
    
    t_list.append(t)
    mean_list.append(mean)
    std_list.append(std)
    visit_list.append(visit)
    
    
    
#     fig, ax = plt.subplots(1,2, figsize = (20,6))
    
    
#     print(mean, std)
#     ax[0].set_title("Gaussian")
#     ax[0].plot(x[3:-2], c = "r")
#     ax[0].plot(x_gauss[3:-2], c = "k")
#     ax[1].set_title("Low Pass")
#     ax[1].plot(x[3:-2], 'r')
#     ax[1].plot(x_filtered[3:-2], 'k')
#     plt.savefig(figpath + "\\" + element[81:-4] + ".png")
    plt.show()

C:\Users\Daniellab\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


Experimental_1_52523
459
Experimental_4_66232
352
Experimental_4_72340
401
Experimental_7_36526
405
Experimental_7_40443
257
Experimental_7_41552
340
Experimental_7_54607
335
Sham_10_45293
272
Sham_11_98276
361
Sham_14_23923
278
Sham_15_48082
247
Sham_16_31431
251
Sham_18_60894
248
Sham_1_32201
420
Sham_1_33803
1523
Sham_1_35598
299
Sham_1_37142
560
Sham_1_38514
353
Sham_2_46690
287
Sham_2_58256
256
Sham_3_77208
286
Sham_4_66374
221
Sham_4_67307
328
Sham_6_153286
548
Sham_6_83460
267
Sham_7_63837
226
Sham_7_64994
440


In [50]:
e

28

In [10]:
#feed this into the mixed effects model in R
name_list = []
for element in csv_list:
    name_list.append(element[81:-4])

In [11]:
name_list_ = []
for name in name_list:
    if name.startswith('Sham'):
        name = name[:7]
        if name.endswith('_'):
            name = name[:-1]
    elif name.startswith('Experimental'):
        name = name[:15]
        if name.endswith('_'):
            name = name[:-1]
    
    name_list_.append(name)
#     print(name)
name_list = name_list_

In [12]:
len(mean_list),len(std_list), len(name_list), len(visit_list)

(56, 56, 56, 56)

In [13]:
condition = ["Experimental"]*13
condition.extend(["Sham"]*43)
new_df = (pd.DataFrame({'name' : name_list, 
                        'std': std_list, 'mean' : mean_list, 'tortuosity_ratio': t_list,
                        'condition': condition, 'visit_number': visit_list}))
new_df = new_df.sort_values(by=['name','visit_number'])
new_df.reset_index(drop=True, inplace=True)

In [14]:
new_df.to_csv(outpath + "\\" + "tortuosity.csv")